<a href="https://colab.research.google.com/github/laymond1/XAI-CL-project/blob/master/baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install avalanche-lib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 582 kB 14.9 MB/s 
     |████████████████████████████████| 1.9 MB 55.0 MB/s 
     |████████████████████████████████| 532 kB 44.9 MB/s 
     |████████████████████████████████| 427 kB 61.5 MB/s 
     |████████████████████████████████| 529 kB 65.3 MB/s 
     |████████████████████████████████| 168 kB 70.6 MB/s 
     |████████████████████████████████| 182 kB 72.8 MB/s 
     |████████████████████████████████| 62 kB 1.6 MB/s 
     |████████████████████████████████| 168 kB 61.1 MB/s 
     |████████████████████████████████| 166 kB 74.1 MB/s 
     |████████████████████████████████| 166 kB 71.2 MB/s 
     |████████████████████████████████| 162 kB 76.6 MB/s 
     |████████████████████████████████| 162 kB 74.7 MB/s 
     |████████████████████████████████| 158 kB 75.4 MB/s 
     |████████████████████████████████| 157 kB 72.9 MB/s 
     |███████████████████████████

In [ ]:
import avalanche
avalanche.__version__

'0.2.1'

In [ ]:
!python --version

Python 3.7.15


In [ ]:
# basics
import numpy as np
import torch

from torch.optim.lr_scheduler import MultiStepLR
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from torchvision import transforms

# Avalanche
# Dataset
from avalanche.benchmarks.classic import CORe50, SplitTinyImageNet, SplitCIFAR10, \
    SplitCIFAR100, SplitCIFAR110, SplitMNIST, RotatedMNIST, PermutedMNIST, SplitCUB200
# Models
from avalanche.models import SimpleMLP, IcarlNet, make_icarl_net, initialize_icarl_net
# training
from avalanche.training import Naive, ICaRL
from avalanche.training.plugins.lr_scheduling import LRSchedulerPlugin
from avalanche.training.plugins import EvaluationPlugin

# Evaluation Metrics
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics,\
    loss_metrics, timing_metrics, cpu_usage_metrics, StreamConfusionMatrix,\
    disk_usage_metrics, gpu_usage_metrics
# from avalanche.evaluation.metrics import *
# Logger
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger

In [ ]:
from types import SimpleNamespace
import torch
import numpy as np
import random

# set_seed, create_default_args

def set_seed(seed):
    if seed is None:
        return
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.enabled = True
        torch.backends.cudnn.benchmark = False


def create_default_args(args_dict, additional_args=None):
    args = SimpleNamespace()
    for k, v in args_dict.items():
        args.__dict__[k] = v
    if additional_args is not None:
        for k, v in additional_args.items():
            args.__dict__[k] = v
    return args

In [ ]:
def icarl_cifar100_augment_data(img):
    img = img.numpy()
    padded = np.pad(img, ((0, 0), (4, 4), (4, 4)), mode='constant')
    random_cropped = np.zeros(img.shape, dtype=np.float32)
    crop = np.random.randint(0, high=8 + 1, size=(2,))

    # Cropping and possible flipping
    if np.random.randint(2) > 0:
        random_cropped[:, :, :] = \
            padded[:, crop[0]:(crop[0]+32), crop[1]:(crop[1]+32)]
    else:
        random_cropped[:, :, :] = \
            padded[:, crop[0]:(crop[0]+32), crop[1]:(crop[1]+32)][:, :, ::-1]
    t = torch.tensor(random_cropped)
    return t


def icarl_scifar100(override_args=None):
    """
    "iCaRL: Incremental Classifier and Representation Learning",
    Sylvestre-Alvise Rebuffi, Alexander Kolesnikov, Georg Sperl, Christoph H. Lampert;
    Proceedings of the IEEE Conference on
    Computer Vision and Pattern Recognition (CVPR), 2017, pp. 2001-2010
    https://openaccess.thecvf.com/content_cvpr_2017/html/Rebuffi_iCaRL_Incremental_Classifier_CVPR_2017_paper.html
    """
    args = create_default_args({'cuda': 0, 'batch_size': 128, 'nb_exp': 10,
                                'memory_size': 2000, 'epochs': 10, 'lr_base': 2.,
                                'lr_milestones': [49, 63], 'lr_factor': 5.,
                                'wght_decay': 0.00001, 'train_mb_size': 256,
                                'seed': None}, override_args)
    # class incremental learning: classes mutual exclusive
    fixed_class_order = [87, 0, 52, 58, 44, 91, 68, 97, 51, 15,
                         94, 92, 10, 72, 49, 78, 61, 14, 8, 86,
                         84, 96, 18, 24, 32, 45, 88, 11, 4, 67,
                         69, 66, 77, 47, 79, 93, 29, 50, 57, 83,
                         17, 81, 41, 12, 37, 59, 25, 20, 80, 73,
                         1, 28, 6, 46, 62, 82, 53, 9, 31, 75,
                         38, 63, 33, 74, 27, 22, 36, 3, 16, 21,
                         60, 19, 70, 90, 89, 43, 5, 42, 65, 76,
                         40, 30, 23, 85, 2, 95, 56, 48, 71, 64,
                         98, 13, 99, 7, 34, 55, 54, 26, 35, 39]
    set_seed(args.seed)
    device = torch.device(f"cuda:{args.cuda}"
                          if torch.cuda.is_available() and
                             args.cuda >= 0 else "cpu")

    benchmark = SplitCIFAR100(n_experiences=args.nb_exp, seed=args.seed,
                  fixed_class_order=fixed_class_order)

    # log to Tensorboard
    tb_logger = TensorboardLogger()

    # log to text file
    text_logger = TextLogger(open('icarl_scifar_log.txt', 'a'))

    # standard log
    interactive_logger = InteractiveLogger()

    eval_plugin = EvaluationPlugin(
        accuracy_metrics(epoch=True, experience=True, stream=True),
        loss_metrics(epoch=True, experience=True, stream=True),
        forgetting_metrics(experience=True, stream=True),
        disk_usage_metrics(epoch=True, experience=True, stream=True),
        loggers = [interactive_logger, text_logger, tb_logger],
        benchmark = benchmark
        )

    # _____________________________Strategy
    model: IcarlNet = make_icarl_net(num_classes=100)
    model.apply(initialize_icarl_net)

    optim = SGD(model.parameters(), lr=args.lr_base,
                weight_decay=args.wght_decay, momentum=0.9)
    sched = LRSchedulerPlugin(
        MultiStepLR(optim, args.lr_milestones, gamma=1.0 / args.lr_factor))

    strategy = ICaRL(
        model.feature_extractor, model.classifier, optim,
        args.memory_size,
        buffer_transform=transforms.Compose([icarl_cifar100_augment_data]),
        fixed_memory=True, train_mb_size=args.batch_size,
        train_epochs=args.epochs, eval_mb_size=args.batch_size,
        plugins=[sched], device=device, evaluator=eval_plugin
    )
    # Dict to iCaRL Evaluation Protocol: Average Incremental Accuracy
    dict_iCaRL_aia = {}
    # ___________________________________________train and eval
    for i, exp in enumerate(benchmark.train_stream):
        strategy.train(exp, num_workers=0)
        res = strategy.eval(benchmark.test_stream[:i + 1], num_workers=0)
        dict_iCaRL_aia['Top1_Acc_Stream/Exp'+str(i)] = res['Top1_Acc_Stream/eval_phase/test_stream/Task000']

    return dict_iCaRL_aia


In [ ]:
res_scifar100 = icarl_scifar100()
print(res_scifar100)

Files already downloaded and verified
Files already downloaded and verified
-- >> Start of training phase << --
100%|██████████| 40/40 [30:21<00:00, 45.53s/it]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 29125508.3652
	Loss_Epoch/train_phase/train_stream/Task000 = 0.0687
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.2190
100%|██████████| 40/40 [26:48<00:00, 40.22s/it]
Epoch 1 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 29125509.0771
	Loss_Epoch/train_phase/train_stream/Task000 = 0.0245
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.4426
100%|██████████| 40/40 [25:15<00:00, 37.89s/it]
Epoch 2 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 29125509.7891
	Loss_Epoch/train_phase/train_stream/Task000 = 0.0206
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.5600
100%|██████████| 40/40 [29:23<00:00, 44.09s/it]
Epoch 3 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 29125510.5010
	Loss_Epoch/train_phase/train_stream

In [ ]:
# For MNIST
# def icarl_mnist_augment_data(img):
#     img = img.numpy()
#     padded = np.pad(img, ((0, 0), (4, 4), (4, 4)), mode='constant')
#     random_cropped = np.zeros(img.shape, dtype=np.float32)
#     crop = np.random.randint(0, high=8 + 1, size=(2,))

#     # Cropping and possible flipping
#     if np.random.randint(2) > 0:
#         random_cropped[:, :, :] = \
#             padded[:, crop[0]:(crop[0]+32), crop[1]:(crop[1]+32)]
#     else:
#         random_cropped[:, :, :] = \
#             padded[:, crop[0]:(crop[0]+32), crop[1]:(crop[1]+32)][:, :, ::-1]
#     t = torch.tensor(random_cropped)
#     return t


def icarl_pmnist(override_args=None):
    """
    "iCaRL: Incremental Classifier and Representation Learning",
    Sylvestre-Alvise Rebuffi, Alexander Kolesnikov, Georg Sperl, Christoph H. Lampert;
    Proceedings of the IEEE Conference on
    Computer Vision and Pattern Recognition (CVPR), 2017, pp. 2001-2010
    https://openaccess.thecvf.com/content_cvpr_2017/html/Rebuffi_iCaRL_Incremental_Classifier_CVPR_2017_paper.html
    """
    args = create_default_args({'cuda': 0, 'batch_size': 128, 'nb_exp': 5,
                                'memory_size': 200, 'epochs': 10, 'lr_base': 2.,
                                'lr_milestones': [10], 'lr_factor': 5.,
                                'wght_decay': 0.00001, 'train_mb_size': 256,
                                'seed': 42}, override_args)
    # class incremental learning: classes mutual exclusive
    fixed_class_order = [3, 0, 1, 8, 9, 5, 4, 2, 6, 7]
    set_seed(args.seed)
    device = torch.device(f"cuda:{args.cuda}"
                          if torch.cuda.is_available() and
                             args.cuda >= 0 else "cpu")

    benchmark = PermutedMNIST(n_experiences=args.nb_exp, seed=args.seed)

    # log to Tensorboard
    tb_logger = TensorboardLogger()

    # log to text file
    text_logger = TextLogger(open('icarl_pmnist_log.txt', 'a'))

    # standard log
    interactive_logger = InteractiveLogger()

    eval_plugin = EvaluationPlugin(
        accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
        loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
        forgetting_metrics(experience=True, stream=True),
        disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
        loggers = [interactive_logger, text_logger, tb_logger],
        benchmark = benchmark
        )

    # _____________________________Strategy
    model: IcarlNet = make_icarl_net(num_classes=10)
    model.apply(initialize_icarl_net)

    optim = SGD(model.parameters(), lr=args.lr_base,
                weight_decay=args.wght_decay, momentum=0.9)
    sched = LRSchedulerPlugin(
        MultiStepLR(optim, args.lr_milestones, gamma=1.0 / args.lr_factor))

    strategy = ICaRL(
        model.feature_extractor, model.classifier, optim,
        args.memory_size,
        buffer_transform=transforms.Compose([icarl_cifar100_augment_data]),
        fixed_memory=True, train_mb_size=args.batch_size,
        train_epochs=args.epochs, eval_mb_size=args.batch_size,
        plugins=[sched], device=device, evaluator=eval_plugin
    )
    # Dict to iCaRL Evaluation Protocol: Average Incremental Accuracy
    dict_iCaRL_aia = {}
    # ___________________________________________train and eval
    for i, exp in enumerate(benchmark.train_stream):
        strategy.train(exp, num_workers=4)
        res = strategy.eval(benchmark.test_stream[:i + 1], num_workers=4)
        dict_iCaRL_aia['Top1_Acc_Stream/Exp'+str(i)] = res['Top1_Acc_Stream/eval_phase/test_stream/Task000']

    return dict_iCaRL_aia


In [ ]:
res_pmnist = icarl_pmnist()
print(res_pmnist)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

...

  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw

  0%|          | 0/55 [11:42<?, ?it/s]
-- >> Start of training phase << --


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


0it [00:00, ?it/s]

RuntimeError: ignored

In [ ]:
# creating the benchmark (scenario object)
benchmark = PermutedMNIST(
    n_experiences=5, 
    seed=42)

# MODEL CREATION
model = SimpleMLP(num_classes=benchmark.n_classes)

# log to Tensorboard
tb_logger = TensorboardLogger()

# log to text file
text_logger = TextLogger(open('log.txt', 'a'))

# standard log
interactive_logger = InteractiveLogger()

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    forgetting_metrics(experience=True, stream=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers = [interactive_logger, text_logger, tb_logger],
    benchmark = benchmark
)

In [ ]:
# CREATE THE STRATEGY INSTANCE (NAIVE)
cl_strategy = Naive(
    model, SGD(model.parameters(), lr=0.001, momentum=0.9), CrossEntropyLoss(),
    train_mb_size=512, train_epochs=1, eval_mb_size=256,
    evaluator=eval_plugin
)

# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in benchmark.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience, num_workers=4)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # eval also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(benchmark.test_stream, num_workers=4))

In [ ]:
from torch.utils.data import DataLoader

class MyStrategy():
    """My Basic Strategy"""

    def __init__(self, model, optimizer, criterion):
        self.model = model
        self.optimizer = optimizer
        self.criterion = criterion

    def train(self, experience):
        # here you can implement your own training loop for each experience (i.e. 
        # batch or task).

        train_dataset = experience.dataset
        t = experience.task_label
        train_data_loader = DataLoader(
            train_dataset, num_workers=4, batch_size=128
        )

        for epoch in range(1):
            for mb in train_data_loader:
                # you code here... XAI
                pass

    def eval(self, experience):
        # here you can implement your own eval loop for each experience (i.e. 
        # batch or task).

        eval_dataset = experience.dataset
        t = experience.task_label
        eval_data_loader = DataLoader(
            eval_dataset, num_workers=4, batch_size=128
        )

        # eval here